# Nulls, Errors, and Exceptions - The Scala Way

These three concepts are amongst the most difficult to deal with in Java.  They are the source of many bugs.  But Scala has some tools to help deal with them, increasing overall quality and reducing bugs related to them.

### --- Nulls ---

If you have any function, can you tell from it's type signature if it might return a null?  Do you need to write code and tests to check for nulls?  They are probably the single biggest source of runtime exceptions, because developers do not realize a function might return a null or forget to check for it.

In Scala, we try to avoid nulls completely.  We have an alternative, the __[Option](https://www.scala-lang.org/api/2.12.8/scala/Option.html)__ class (side note - Java 8 borrowed this concept for it's `Optional` class).  Of the three classes we will be discussing, this one is the simplest.  It has a type signature that takes one parameter and looks like `Option[A]`.  A null is represented by a `None[A]`, which holds zero elements of type A.  Any non-null value is represented by a `Some[A]`, which holds one element of type A, namely the value.

In [1]:
val option1: Option[String] = None
val option2: Option[String] = Some("foobar")

option1: Option[String] = None
option2: Option[String] = Some("foobar")

An option can also be constructed by in the following way, based on a variable that may or may not be null.  This is useful for wrapping functions which are known to possibly return a null value.

In [2]:
val optionVar1: String = null
val optionVar2: String = "foobar"

Option(optionVar1)
Option(optionVar2)

optionVar1: String = null
optionVar2: String = "foobar"
res1_2: Option[String] = None
res1_3: Option[String] = Some("foobar")

You can check to see if an `Option` is a `None` or a `Some` with the `isEmpty` method.  This is a more idiomatic way of doing it, versus a `match` and `case` statements.

In [3]:
option1.isEmpty
option2.isEmpty

res2_0: Boolean = true
res2_1: Boolean = false

You can use the `get` method to extract the value from a `Some`, but will raise an exception for a `None`.  A better alternative is the `getOrElse` method, which extracts the value from a `Some`, and returns a specified default value for a `None`.  Note that the type of the argument must match the inner type of the `Option`.

In [4]:
option1.getOrElse("unknown")
option2.getOrElse("unknown")

res3_0: String = "unknown"
res3_1: String = "foobar"

If you have one or more transformations that always return a value, you can chain together calls to the `map` method.  It takes a function with the type signature `A => B`.  Note that in the case of the `Null[A]`, the result is a `None[B]`.  The transformation is applied to any value held within the `Option`, which for a `Null` is empty.

In [5]:
def valueFunction1(x: String): Int = x.size

option1.map(valueFunction1)
option2.map(valueFunction1)

defined function valueFunction1
res4_1: Option[Int] = None
res4_2: Option[Int] = Some(6)

If you have one or more transformations that each return an Option, you can chain together calls to the `flatMap` method.  It takes a function with the type signature `A => Option[B]`.  The "map" part returns an `Option[Option[B]]`, and the "flat" part takes that and returns just a `Option[B]`.  It is identical as calling the `map` method followed by the `flatten` method.

In [6]:
def optionFunction1Pass(x: String): Option[Int] = Some(x.size)
def optionFunction2Pass(x: Int): Option[Int] = Some(x * 3)
def optionFunction3Pass(x: Int): Option[Double] = Some(x.toDouble)
def optionFunction4Pass(x: Double): Option[String] = Some(x.toString)

option2
  .flatMap(optionFunction1Pass)
  .flatMap(optionFunction2Pass)
  .flatMap(optionFunction3Pass)
  .flatMap(optionFunction4Pass)

defined function optionFunction1Pass
defined function optionFunction2Pass
defined function optionFunction3Pass
defined function optionFunction4Pass
res5_4: Option[String] = Some("18.0")

If at any point a transformation returns a `None`, all the subsequent `flatMap` calls will also return a `None`, but the type will change with each transformation.

In [7]:
def optionFunction3Fail(x: Int): Option[Double] = None

option2
  .flatMap(optionFunction1Pass)
  .flatMap(optionFunction2Pass)
  .flatMap(optionFunction3Fail)
  .flatMap(optionFunction4Pass)

defined function optionFunction3Fail
res6_1: Option[String] = None

This is the power of functional programming - composition.  But here we are composing together functions which may (only conceptually now) return nulls.  Without `Option`, for N transformations, you would have to write an N-level deep nested `if` structure, checking for nulls at each level.  This is not readable or easily maintained code.

Of course, `map` and `flatMap` can be mixed together in a sequence of calls where some transformations return a value and some return an `Option`.

There are a number of other useful methods on `Option`, like `contains`, `exists`, `filter`, etc.  Please refer to the documentation linked above.  It can also be converted to an `Either`, as a `Left` or `Right` - discussed further below.

In [8]:
option1.contains("blah")
option2.contains("blah")
option2.contains("foobar")

res7_0: Boolean = false
res7_1: Boolean = false
res7_2: Boolean = true

In [9]:
option1.exists(_.size == 5)
option2.exists(_.size == 5)
option2.exists(_.size == 6)

res8_0: Boolean = false
res8_1: Boolean = false
res8_2: Boolean = true

In [10]:
option1.filter(_ == "blah")
option2.filter(_ == "blah")
option2.filter(_ == "foobar")

res9_0: Option[String] = None
res9_1: Option[String] = None
res9_2: Option[String] = Some("foobar")

In [11]:
case class Thing(id: Int)

option1.toLeft(Thing(1))
option2.toLeft(Thing(2))

option1.toRight(Thing(1))
option2.toRight(Thing(2))

defined class Thing
res10_1: Either[String, Thing] = Right(Thing(1))
res10_2: Either[String, Thing] = Left("foobar")
res10_3: Either[Thing, String] = Left(Thing(1))
res10_4: Either[Thing, String] = Right("foobar")

### --- Errors ---

In Scala, we usually deal with a method that can return a value or some kind of error using the __[Either](https://www.scala-lang.org/api/2.12.8/scala/util/Either.html)__ class.  It has a type signature that takes two parameters and looks like `Either[A, B]`.  A failure is represented by a `Left[A]`. A Success is represented by a `Right[B]`.

In [12]:
case class Error(message: String)

val either1: Either[Error, Int] = Left(Error("error"))
val either2: Either[Error, Int] = Right(123)

defined class Error
either1: Either[Error, Int] = Left(Error("error"))
either2: Either[Error, Int] = Right(123)

Either is right biased, meaning operators like `map` and `flatMap` only operate on a Right.  For example, the type signature for map is `map[C](B => C): Either[A,C]`.  Only the value within a `Right` will be operated on.  The value within a `Left` will be unchanged.  Either way, the type signature of the `Either` will changed.

In [13]:
either1.map(_.toDouble)
either2.map(_.toDouble)

res12_0: Either[Error, Double] = Left(Error("error"))
res12_1: Either[Error, Double] = Right(123.0)

blah blah

In [14]:
def eitherFunc1Pass(x: Int): Either[Error, Int] = Right(x + 321)
def eitherFunc2Pass(x: Int): Either[Error, Int] = Right(x * 2)
def eitherFunc3Pass(x: Int): Either[Error, Double] = Right(x.toDouble)
def eitherFunc4Pass(x: Double): Either[Error, String] = Right(x.toString)

either2
  .flatMap(eitherFunc1Pass)
  .flatMap(eitherFunc2Pass)
  .flatMap(eitherFunc3Pass)
  .flatMap(eitherFunc4Pass)

defined function eitherFunc1Pass
defined function eitherFunc2Pass
defined function eitherFunc3Pass
defined function eitherFunc4Pass
res13_4: Either[Error, String] = Right("888.0")

In [15]:
def eitherFunc3Fail(x: Int): Either[Error, Double] = Left(Error("function 3 failed"))

either2
  .flatMap(eitherFunc1Pass)
  .flatMap(eitherFunc2Pass)
  .flatMap(eitherFunc3Fail)
  .flatMap(eitherFunc4Pass)

defined function eitherFunc3Fail
res14_1: Either[Error, String] = Left(Error("function 3 failed"))

### --- Exceptions ---

For the situation where a function might throw an exception, we have the `Try` class, which operates very similarly to the `Either` class.  It takes a single type parameter and has the type signature of `Try[A]`. failure is represented by a `Failure[A]`, which actually holds an exception of type `Throwable`.  A success is represented by a `Success[A]` and holds the returned value.

In [16]:
import scala.util.Try

val try1 = Try(3 / 0)
val try2 = Try(3 / 1)

import scala.util.Try


try1: Try[Int] = Failure(java.lang.ArithmeticException: / by zero)
try2: Try[Int] = Success(3)